<a href="https://colab.research.google.com/github/Haeshitha4567/Movie-Recommendation/blob/main/movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
uploaded=files.upload()


Saving tmdb_5000_credits.csv to tmdb_5000_credits.csv


In [ ]:
import pandas as pd
import io
df1=pd.read_csv(io.BytesIO(uploaded['tmdb_5000_credits.csv']))
print(df1)

      movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4798      9367                               El Mariachi   
4799     72766                                 Newlyweds   
4800    231617                 Signed, Sealed, Delivered   
4801    126186                          Shanghai Calling   
4802     25975                         My Date with Drew   

                                                   cast  \
0     [{"cast_id": 242, "character": "Jake Sully", "...   
1     [{"cast_id": 4, "character": "Captain Jack Spa...   
2     [{"cast_id": 1, "character": "James Bond", "cr...   
3     [{"cast_id": 2, "character": "Bruce W

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving tmdb_5000_movies.csv to tmdb_5000_movies.csv


In [ ]:
df2=pd.read_csv(io.BytesIO(uploaded['tmdb_5000_movies.csv']))
print(df2)

         budget                                             genres  \
0     237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1     300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2     245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3     250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4     260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
...         ...                                                ...   
4798     220000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4799       9000  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   
4800          0  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...   
4801          0                                                 []   
4802          0                [{"id": 99, "name": "Documentary"}]   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   19995   
1          http://disney.

In [ ]:
final=pd.merge(df1,df2)
print(final)

      movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4804      9367                               El Mariachi   
4805     72766                                 Newlyweds   
4806    231617                 Signed, Sealed, Delivered   
4807    126186                          Shanghai Calling   
4808     25975                         My Date with Drew   

                                                   cast  \
0     [{"cast_id": 242, "character": "Jake Sully", "...   
1     [{"cast_id": 4, "character": "Captain Jack Spa...   
2     [{"cast_id": 1, "character": "James Bond", "cr...   
3     [{"cast_id": 2, "character": "Bruce W

In [ ]:
final=final[["overview","original_title","id","genres","original_language"]]

In [ ]:
final.isnull().sum()

overview             3
original_title       0
id                   0
genres               0
original_language    0
dtype: int64

In [ ]:
final.dropna(inplace=True)

In [ ]:
final.duplicated().value_counts()

False    4800
True        6
dtype: int64

In [ ]:
final=final.drop_duplicates(keep="first",inplace=False)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfv=TfidfVectorizer(min_df=3,max_features =None, strip_accents= "unicode",analyzer='word',token_pattern=r'\w{1,}',
                    ngram_range=(1,3),stop_words='english')

In [ ]:
tfv_matrix=tfv.fit_transform(final["overview"])

In [ ]:
tfv_matrix.shape

(4800, 10417)

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig=sigmoid_kernel(tfv_matrix,tfv_matrix)
sig

array([[0.76163447, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
        0.76159416],
       [0.76159416, 0.76163447, 0.76159416, ..., 0.76159513, 0.76159416,
        0.76159416],
       [0.76159416, 0.76159416, 0.76163447, ..., 0.76159486, 0.76159416,
        0.76159455],
       ...,
       [0.76159416, 0.76159513, 0.76159486, ..., 0.76163447, 0.76159483,
        0.76159473],
       [0.76159416, 0.76159416, 0.76159416, ..., 0.76159483, 0.76163447,
        0.76159461],
       [0.76159416, 0.76159416, 0.76159455, ..., 0.76159473, 0.76159461,
        0.76163447]])

In [ ]:
index1=pd.Series(final.index,index=final["original_title"]).drop_duplicates()
index1

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4804
Newlyweds                                   4805
Signed, Sealed, Delivered                   4806
Shanghai Calling                            4807
My Date with Drew                           4808
Length: 4800, dtype: int64

In [ ]:
def give_recommendation(title,sig=sig):
    idx=index1[title]
    sigmoid_score= list(enumerate(sig[idx]))
    sigmoid_score=sorted(sigmoid_score, key=lambda x:x[1],reverse=True)
    sigmoid_score=sigmoid_score[1:6]
    movie_indices=[i[0] for i in sigmoid_score]
    return final["original_title"].iloc[movie_indices]

In [ ]:
give_recommendation("Spectre")

1346    Never Say Never Again
29                    Skyfall
3167              Thunderball
4078    From Russia with Love
11          Quantum of Solace
Name: original_title, dtype: object